# Trabalho Presencial - AV1

**Aluno**: *Guilherme Massahiro Takemori*

## Preparação de ambiente

A célula abaixo instala e importa as bibliotecas necessárias para a realização desse trabalho.

Retire os comentários da instalação caso o seu ambiente não possua as bibliotecas.

In [4]:
#! pip install altair
#! pip install pandas
#! pip install numpy
#! pip install nbconvert

import altair as alt
import pandas as pd
import numpy as np
import nbconvert

Devido ao tamanho da base de dados, será necessário utilizar as bibliotecas abaixo:

In [5]:
! pip install "vegafusion[embed]>=1.5.0"
! pip install "vl-convert-python>=1.6.0"


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: C:\Users\Gabriel\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: C:\Users\Gabriel\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


Vou escolher a base de dados de **Alfabetização por Sexo, Raça e Grupo de Idade** ([Link da base](https://basedosdados.org/dataset/08a1546e-251f-4546-9fe0-b1e6ab2b203d?table=cf9537b5-6198-455f-a8b0-7c762e94d79c)):

In [6]:
dado = pd.read_csv('br_ibge_censo_2022_alfabetizacao_grupo_idade_sexo_raca.csv', on_bad_lines='skip')
dado

,id_municipio,cor_raca,sexo,grupo_idade,alfabetizacao,populacao
0,1100023,Indígena,Homens,15 a 19 anos,Não alfabetizadas,NaN
1,1100262,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,NaN
2,1101005,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,NaN
3,1101435,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,NaN
4,1101435,Indígena,Mulheres,15 a 19 anos,Não alfabetizadas,NaN
...,...,...,...,...,...,...
779795,2924009,Parda,Homens,65 anos ou mais,Alfabetizadas,1791.0
779796,4318903,Branca,Mulheres,65 anos ou mais,Alfabetizadas,2047.0
779797,4127700,Branca,Homens,65 anos ou mais,Alfabetizadas,4351.0
779798,3302700,Branca,Mulheres,65 anos ou mais,Alfabetizadas,7167.0


Temos assim, as variáveis com seus respectivos tipos:

* `id_municipio`: nominal
* `cor_raca`: nominal
* `sexo`: nominal
* `grupo_idade`: ordinal
* `alfabetizacao`: nominal
* `populacao`: quantitativa

Podemos perceber que a base possui problemas:


In [7]:
print(list(pd.unique(dado[~pd.to_numeric(dado['id_municipio'], errors='coerce').notna()]['id_municipio'])), '\n') # Dados não numéricos ou nulos na coluna id_municipio
print(pd.unique(dado['cor_raca']), '\n')
print(pd.unique(dado['sexo']), '\n')
print(pd.unique(dado['grupo_idade']), '\n')
print(pd.unique(dado['alfabetizacao']), '\n')
print(list(pd.unique(dado[~pd.to_numeric(dado['populacao'], errors='coerce').notna()]['populacao']))) # Dados não numéricos ou nulos na coluna populacao

[] 

['Indígena' 'Amarela' 'Preta' 'Branca' 'Parda'] 

['Homens' 'Mulheres'] 

['15 a 19 anos' '20 a 24 anos' '25 a 34 anos' '35 a 44 anos'
 '45 a 54 anos' '55 a 64 anos' '65 anos ou mais'] 

['Não alfabetizadas' 'Alfabetizadas'] 

[nan]


Portanto limpemos-na:

In [8]:
dado = dado[pd.to_numeric(dado['id_municipio'], errors='coerce').notna() | dado['id_municipio'].isna()]

dado = dado[dado['cor_raca'].isin(('Indígena', 'Amarela', 'Preta', 'Branca', 'Parda')) | dado['cor_raca'].isna()]

dado = dado[dado['sexo'].isin(('Homens', 'Mulheres')) | dado['sexo'].isna()]

dado = dado[dado['grupo_idade'].isin(('15 a 19 anos', '20 a 24 anos', '25 a 34 anos', '35 a 44 anos', '45 a 54 anos', '55 a 64 anos', '65 anos ou mais')) | dado['grupo_idade'].isna()]

dado = dado[dado['alfabetizacao'].isin(('Não alfabetizadas', 'Alfabetizadas')) | dado['alfabetizacao'].isna()]

dado = dado[pd.to_numeric(dado['populacao'], errors='coerce').notna() | dado['populacao'].isna()]

Aqui está os resumos de estatística descritiva:

In [9]:
dado['cor_raca'].value_counts()

cor_raca
Indígena    155960
Amarela     155960
Preta       155960
Branca      155960
Parda       155960
Name: count, dtype: int64

In [10]:
dado['sexo'].value_counts()

sexo
Homens      389900
Mulheres    389900
Name: count, dtype: int64

In [11]:
dado['grupo_idade'].value_counts()

grupo_idade
15 a 19 anos       111400
20 a 24 anos       111400
25 a 34 anos       111400
35 a 44 anos       111400
45 a 54 anos       111400
55 a 64 anos       111400
65 anos ou mais    111400
Name: count, dtype: int64

In [12]:
dado['alfabetizacao'].value_counts()

alfabetizacao
Não alfabetizadas    389900
Alfabetizadas        389900
Name: count, dtype: int64

In [13]:
dado.describe()

,id_municipio,populacao
count,7.798000e+05,529715.000000
mean,3.253591e+06,307.603172
std,9.848226e+05,3154.010564
min,1.100015e+06,1.000000
25%,2.512101e+06,5.000000
50%,3.146280e+06,30.000000
75%,4.119202e+06,139.000000
max,5.300108e+06,536242.000000


Aqui está a análise bidimensional para um par de variáveis categóricas:

In [14]:
alt.data_transformers.enable("vegafusion")
alt.Chart(dado).mark_bar().encode(
    x=alt.X('cor_raca:N'),
    y=alt.Y('populacao:Q'),
    color=alt.Color('alfabetizacao:N')
).interactive()

TypeError: Expected Table-like input or dict of arrays or sequence of arrays.

alt.Chart(...)

Me parece que as raças parda e preta tendem à serem menos favorecidas em termos de alfabetização, do que a raça branca.

In [15]:
proporcoes = dado.groupby(['cor_raca', 'alfabetizacao']).agg({'populacao': 'sum'}).reset_index()
proporcoes = proporcoes[proporcoes['alfabetizacao'] == 'Alfabetizadas'].reset_index()
populacao_total = dado.groupby('cor_raca')['populacao'].sum().reset_index()

proporcoes['proporcao'] = proporcoes['populacao'] / populacao_total['populacao']

alt.Chart(proporcoes).mark_bar().encode(
    x=alt.X('cor_raca:N'),
    y=alt.Y('proporcao:Q')
).interactive()

TypeError: Expected Table-like input or dict of arrays or sequence of arrays.

alt.Chart(...)

Para o par de variáveis quantitativas, vou usar outra base de dados ([link da base de dados](https://basedosdados.org/dataset/c990253a-8e63-4880-9d45-bca2559aa8e3?table=e2b07b3b-71e4-483b-aca6-4d11e9160b93)):

In [16]:
dado2 = pd.read_csv('nl_ug_pwt_microdados.csv')
dado2

,year,country_id_iso_3,country,currency_unit,population,employees,real_gdp_expenditures,real_gdp_output,current_consumption,current_absorption,...,indicator_real_rate_return,indicator_stat_capacity,corr_exp_share,current_share_household,current_share_gcf,current_share_gov,current_share_x,current_share_m,current_share_residual_trade,current_share_labour_gdp_na
0,1950,ABW,Aruba,Aruban Guilder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951,ABW,Aruba,Aruban Guilder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1952,ABW,Aruba,Aruban Guilder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1953,ABW,Aruba,Aruban Guilder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1954,ABW,Aruba,Aruban Guilder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12805,2015,ZWE,Zimbabwe,US Dollar,13814629.0,6.393752e+06,4.014162e+10,3.979864e+10,4.424936e+10,4.732546e+10,...,Regular,62.222200,0.306506,0.945086,0.077963,0.176403,0.140172,-0.287693,-0.051930,0.533381
12806,2016,ZWE,Zimbabwe,US Dollar,14030331.0,6.504374e+06,4.187520e+10,4.096319e+10,4.329157e+10,4.640416e+10,...,Regular,55.555533,0.302841,0.890514,0.076169,0.168887,0.131920,-0.251232,-0.016258,0.533381
12807,2017,ZWE,Zimbabwe,US Dollar,14236595.0,6.611773e+06,4.467218e+10,4.431674e+10,4.611395e+10,4.945757e+10,...,Regular,59.999967,0.301868,0.833453,0.075448,0.207101,0.126722,-0.202827,-0.039897,0.533381
12808,2018,ZWE,Zimbabwe,US Dollar,14438802.0,6.714952e+06,4.432511e+10,4.342090e+10,4.619844e+10,4.966539e+10,...,Regular,59.999967,NaN,0.790588,0.079576,0.269799,0.144485,-0.263658,-0.020791,0.533381


In [17]:
barra_ano = alt.selection_point(
    name='Ano: ',
    fields=['year'],
    bind=alt.binding_range(
        min=1950,
        max=2019,
        step=1,
        name='Ano'
    )
)

barra_pais = alt.selection_point(
    name='País: ',
    fields=['country'],
    bind=alt.binding_select(
        options=pd.unique(dado2['country']),
        name='País'
    )
)

alt.Chart(dado2).mark_line().encode(
    x=alt.X('current_share_gov:Q', title='Participação do Governo no Consumo'),
    y=alt.Y('current_share_household:Q', title='Participação de Moradores no Consumo'),
).add_params(
    barra_ano,
    barra_pais
).transform_filter(
    barra_ano
).transform_filter(
    barra_pais
)

TypeError: Expected Table-like input or dict of arrays or sequence of arrays.

alt.Chart(...)

Podemos observar que não tende à haver muita relação entre as variáveis, exceto em períodos especiais, tipo guerras, onde tende à haver uma relação positiva.

Vou verificar a característica de raça e alfabetização, no seguinte gráfico:

In [18]:
proporcoes = dado.groupby(['cor_raca', 'alfabetizacao']).agg({'populacao': 'sum'}).reset_index()
proporcoes = proporcoes[proporcoes['alfabetizacao'] == 'Alfabetizadas'].reset_index()
populacao_total = dado.groupby('cor_raca')['populacao'].sum().reset_index()

proporcoes['proporcao'] = proporcoes['populacao'] / populacao_total['populacao']

alt.Chart(proporcoes).mark_bar().encode(
    x=alt.X('cor_raca:N'),
    y=alt.Y('proporcao:Q')
).interactive()

TypeError: Expected Table-like input or dict of arrays or sequence of arrays.

alt.Chart(...)

Por fim, podemos ver que de fato há desfavorecimento de raças como indígena, parda e preta em relação à raça branca.

Vimos também, que o consumo do governo não tende à influenciar os gastos de moradores, exeto em períodos especiais.

## Conversão para html

O código abaixo converte o seu notebook em um arquivo do formato `html`. **Lembre de salvar o seu arquivo antes de rodar esse trecho, ou as alterações não aparecerão no html.**

In [19]:
#! jupyter nbconvert --to html AV1.ipynb